# Q-Tensor Relaxation in FEniCSx

Using the Landau-de Gennes free energy with a single-constant approximation: 

$ F_{LDG} = \frac{A}{2}Q_{ij}Q_{ji} + \frac{B}{3} Q_{ij}(Q_{ik}Q_{kj})_{ji} + \frac{C}{4}(Q_{ij}Q_{ji})^2 + \frac{L}{2}\nabla^2Q $

and taking the functional derivative of $F_{LDG}$ to calculate the time derivative.

$ \frac{\partial Q}{\partial t} = - AQ - B Q_{ik}Q{kj} - C(Q_{ij}Q{ji})Q + \frac{L}{2}\nabla^2Q $


$ \frac{Q-Q_0}{\delta t} = - AQ - B Q_{ik}Q{kj} - C(Q_{ij}Q{ji})Q + \frac{L}{2}\nabla^2Q $

I'm not entirely sure that the FEniCSx needs a separate residual and jacobian. I think (accoriding to their documentation) that the non-linear solver can figure out the jacobian by itself.

So to get the weak form of the equation above I do the following:

$F_1 = \frac{Q-Q_0}{\delta t} \Rightarrow F_1 = \int_{\Omega} (\frac{Q-Q_0}{\delta t})V \,dx$

where $V$ is the test function. Likewise, I separate the rest of $F_{LDG}$ into two more terms.

$F_2 =  \int_{\Omega} (-AQ - B Q_{ik}Q{kj} - C(Q_{ij}Q{ji})Q)V\,dx$

$F_3 = \frac{L}{2} \int_{\Omega} (\nabla Q \cdot \nabla V)\,dx$

And then the weak form that goes into FEniCSx will be $F_1 + F_2 + F_3$

In [2]:
import numpy as np
import ufl,time
from dolfinx import *
from mpi4py import MPI
from petsc4py import PETSc
from tqdm.auto import tqdm

In [9]:
axispts = 11
T = 10
nsteps = 100
dt = T/nsteps
isave = False
theta = 1 #time-step family, theta=1 -> backwards Euler, theta=0.5 -> Crank-Nicholson, theta = 0 -> forwards Euler
S0 = 0.53 # order parameter

FEniCSx has a ```TensorElement``` which can be set as symmetric. I used this instead of breaking up the problem into separate values of $Q$ ($q_0,q_1$...) with a ```MixedElement``` function space and having a separate residual for each of them, although I think that might work as well.

In [5]:
# creating a cubic regular mesh from 0.0->1.0 with 11 nodes to each side
msh = mesh.create_unit_cube(comm = MPI.COMM_WORLD,
                                  nx = axispts,
                                  ny = axispts,
                                  nz = axispts)
    
# populating the mesh with a symmetric tensor element at each node
#P = ufl.TensorElement('CG', ufl.tetrahedron, 1, symmetry=True)
P = ufl.TensorElement('CG',msh.ufl_cell(),1,symmetry=True)
# creating the Function space
FS = fem.FunctionSpace(msh,P) #CG == Lagrange

Q = fem.Function(FS) # current time-step result
Q_n = fem.Function(FS) # previous time-step result
V = ufl.TestFunction(FS) # test function to weight calcuations through the lattice

For the initial conditions of the mesh, I want $Q$ to have values corresponding to a random orientation of the directior at each node/point in space. The following function, ```initQ3d``` attempts to do this, following the similar Cahn-Hilliard example in the FEniCSx documentation.

In [6]:
def initQ3d(x):
    # values[0] = tensor[0,0]  --> 0 1 2
    # values[1] = tensor[0,1]      3 4 5
    # values[2] = tensor[0,2]      6 7 8
    # values[3] = tensor[1,0] ...
    values = np.zeros((3*3,
                    x.shape[1]), dtype=np.float64)
    n = np.zeros((3,x[0].shape[0])) # director
    polar_angle = np.arccos(np.random.uniform(-1,1,x[0].shape))
    azi_angle = np.random.uniform(0,2*np.pi)
    n[0,:] = np.sin(polar_angle)*np.cos(azi_angle)
    n[1,:] = np.sin(polar_angle)*np.sin(azi_angle)
    n[2,:] = np.cos(polar_angle)
    #n = np.linalg.norm(n)
    #Qxx = S0*(n[0]*n[0] - 1/3)
    values[0] = S0*(n[0,:]*n[0,:]-1/3)
    values[1] = S0*(n[0,:]*n[1,:])
    values[2] = S0*(n[0,:]*n[2,:])
    values[3] = S0*(n[1,:]*n[0,:])
    values[4] = S0*(n[1,:]*n[1,:]-1/3)
    values[5] = values[3]
    values[6] = values[2]
    values[7] = values[1]
    values[8] = -values[0]-values[4]
    return values

In [7]:
#initializing Q for random director and distributing initial condition
Q.interpolate(initQ3d)

In [10]:
# writing initial conditions to file
if (isave):
    xdmf_Q_file = io.XDMFFile(msh.comm, "qtensor.xdmf",'w')
    xdmf_Q_file.write_mesh(msh)
    xdmf_Q_file.write_function(Q,0.0)
    #vtk_Q_file = io.VTKFile(msh.comm, "qtensor.vtk",'w')
    #vtk_Q_file.write_mesh(msh,0.0)
    print("Initial state written")
    xdmf_Q_file.close()

# defining some constants
A = fem.Constant(msh,PETSc.ScalarType(1.0))
B = fem.Constant(msh, PETSc.ScalarType(1.0))
C = fem.Constant(msh, PETSc.ScalarType(1.0))
L = fem.Constant(msh, PETSc.ScalarType(1.0))
k = fem.Constant(msh, PETSc.ScalarType(dt))

# backwards euler part of residual
F1 = ufl.inner((Q - Q_n)/k,V)*ufl.dx 
# bulk free energy part
F2 = -1*ufl.inner((A*Q + B*ufl.dot(Q,Q) + C*(ufl.inner(Q,Q)*Q)),V)*ufl.dx
# distortion/elastic term
F3 = -1*(ufl.inner(ufl.grad(Q),ufl.grad(V)))*ufl.dx
# construct the residual
F = F1+F2+F3
#print(fem.assemble_scalar(F2+F3))
#exit()

The following is almost copied from the Cahn-Hilliard and the Non-linear Poisson equation examples with the exception of changing the convergence criterion.

In [11]:
# Create nonlinear problem and Newton solver
problem = fem.petsc.NonlinearProblem(F, Q)
solver = nls.petsc.NewtonSolver(msh.comm, problem)
solver.convergence_criterion = "residual" #"incremental"
solver.rtol = 1e-6

# We can customize the linear solver used inside the NewtonSolver by
# modifying the PETSc options
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
ksp.setFromOptions()

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_c7c33010deddb6c479364a4dd0c5bce88f7f6cbf' extension
INFO:root:/home/matt/anaconda3/envs/fenics/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/matt/anaconda3/envs/fenics/include -fPIC -O2 -isystem /home/matt/anaconda3/envs/fenics/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/matt/anaconda3/envs/fenics/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/matt/anaconda3/envs/fenics/include -fPIC -I/home/matt/anaconda3/envs/fenics/lib/python3.10/site-packages/ffcx/codegeneration -I/home/matt/anaconda3/envs/fenics/include/python3.10 -c libffcx_forms_c7c33010deddb6c479364a4dd0c5bce88f7f6cbf.c -o ./libffcx_forms_c7c33010deddb6c479364a4dd0c5bce88f7f6cbf.o -O2 -g0
INFO:root:/home/matt/anaconda3/envs/fenics/bin/x86_64-conda-linux-gnu-cc -shared -Wl,--allow-shlib-undefined 

In [12]:
t = 0.0
it = 0
elapsed_time = 0
Q_n.x.array[:] = Q.x.array[:]
while (t < T):
    t += dt
    start_time = time.time()
    r = solver.solve(Q)
    Q_n.x.array[:] = Q.x.array
    if (isave):
        xdmf_Q_file.write_function(Q_n,t)
        #vtk_Q_file.write_function(Q_n,t)
    elapsed_time += time.time()-start_time
    #E = assemble(F2+F3)
    it += r[0]
    print(f"Step {int(t/dt)}/{nsteps}: num iterations: {r[0]}\t {elapsed_time}s,{it/elapsed_time}it/s")

if (isave):
    xdmf_Q_file.close()
print("Done!")
print("Total time: ",elapsed_time)
print("Total steps: ",nsteps)
print("Total iterations: ",it)

Step 1/50: num iterations: 2	 0.656003475189209s,3.0487643368400548it/s
Step 2/50: num iterations: 3	 1.6419000625610352s,3.045252335395494it/s
Step 3/50: num iterations: 30	 10.922283172607422s,3.204458211427659it/s
Step 4/50: num iterations: 13	 14.944137334823608s,3.2119619168747797it/s
Step 5/50: num iterations: 12	 18.677062034606934s,3.2124966918686324it/s
Step 5/50: num iterations: 8	 21.25369381904602s,3.199443850981956it/s
Step 6/50: num iterations: 12	 24.947086811065674s,3.206787253592862it/s
Step 7/50: num iterations: 8	 27.42420983314514s,3.2088435924101786it/s
Step 8/50: num iterations: 12	 31.09834909439087s,3.2156047800632845it/s
Step 9/50: num iterations: 8	 33.58565092086792s,3.215658980511105it/s
Step 10/50: num iterations: 16	 38.568602561950684s,3.2150503716287213it/s
Step 11/50: num iterations: 10	 41.66566228866577s,3.2160775237803376it/s
Step 13/50: num iterations: 12	 45.37142753601074s,3.217884204417452it/s
Step 14/50: num iterations: 15	 49.99348306655884s,3.

The following code snippet converts the way that the Q tensor is written to the xdmf file to a vector vtk file for better visualization without using TensorGlyph filter in Paraview.

In [13]:
import meshio
def xdmf_eig(filename):
    with meshio.xdmf.TimeSeriesReader(filename) as reader:
        points, cells = reader.read_points_cells()
        for k in tqdm(range(reader.num_steps)):
            t, point_data, cell_data = reader.read_data(k)
            data = point_data['f']
            eig_data = np.zeros((data.shape[0],3))
            for p in range(data.shape[0]):
                Q = np.reshape(data[p,:],(3,3))
                w,v = np.linalg.eig(Q) #w gives eigenvalues, v gives eigenvectors (v[:,i])
                eig_data[p,:] = v[:,np.argmax(w)]

            new_mesh = mesh = meshio.Mesh(points,cells,point_data={"N": eig_data})
            vtk_filename = "qtensor"+str(t).replace('.','')+'.vtk'
            mesh.write(vtk_filename)

if (isave):
    print("converting Q-tensor to director field")
    xdmf_eig("qtensor.xdmf")

ModuleNotFoundError: No module named 'meshio'